In [1]:
import os
import openai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
# location of open ai api key
load_dotenv('./key.env')
# read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

In [2]:
from langchain.document_loaders import PyPDFLoader

# Load PDF
loaders = [
    PyPDFLoader("./MachineLearning-Lecture01.pdf"),
    PyPDFLoader("./MachineLearning-Lecture02.pdf"),
    PyPDFLoader("./MachineLearning-Lecture03.pdf")
]
docs = []
for loader in loaders:
    docs.extend(loader.load())

/Users/utkarshdabholkar/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 150
)

In [4]:
splits = text_splitter.split_documents(docs)

In [5]:
from langchain.embeddings.openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings()

/Users/utkarshdabholkar/Library/Python/3.9/lib/python/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. Use langchain_openai.OpenAIEmbeddings instead.
  warn_deprecated(


In [6]:
from langchain.vectorstores import Chroma
persist_directory = 'docs/chroma/'

In [7]:
!rm -rf ./docs/chroma  # remove old database files if any

In [8]:
vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embedding,
    persist_directory=persist_directory
)

In [9]:
# ! pip3 install chromadb==0.4.3

In [10]:
print(vectordb._collection.count())

152


In [11]:
import datetime
current_date = datetime.datetime.now().date()
if current_date < datetime.date(2023, 9, 2):
    llm_name = "gpt-3.5-turbo-0301"
else:
    llm_name = "gpt-3.5-turbo"
print(llm_name)

gpt-3.5-turbo


In [12]:
question = "What are major topics for this class?"
docs = vectordb.similarity_search(question,k=3)
len(docs)

3

In [13]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name=llm_name, temperature=0)

/Users/utkarshdabholkar/Library/Python/3.9/lib/python/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. Use langchain_openai.ChatOpenAI instead.
  warn_deprecated(


In [14]:
from langchain.chains import RetrievalQA

In [15]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

In [16]:
result = qa_chain({"query": question})

/Users/utkarshdabholkar/Library/Python/3.9/lib/python/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [17]:
result["result"]

'The major topics for this class include machine learning, statistics, algebra, linear algebra, and extensions of machine learning.'

### Prompt

In [18]:
from langchain.prompts import PromptTemplate

template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)


In [19]:
# Run chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [20]:
question = "Is probability a class topic?"

In [21]:
result = qa_chain({"query": question})

In [22]:
result["result"]

'Yes, probability is a class topic. Thanks for asking!'

### Integrating memory

In [23]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

### ConversationalRetrievalChain

In [24]:
from langchain.chains import ConversationalRetrievalChain
retriever=vectordb.as_retriever()
qa = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=retriever,
    memory=memory
)

### With this you can ask question to your document with pervious memory

In [26]:
question = "What are top concepts of this class?"
result = qa({"question": question})
result['answer']

'The main topics covered in this class include machine learning, statistics, algebra, linear algebra, and extensions of machine learning.'

In [27]:
question = "Can you explain some of those topics ?"
result = qa({"question": question})
result['answer']

'The topics mentioned in the context are linear algebra, convex optimization, hidden Markov models, and machine learning. \n\nLinear algebra is a branch of mathematics that deals with vector spaces and linear equations. It involves the study of vectors, matrices, and linear transformations. In the context, the instructor mentions that there will be a refresher on linear algebra in the discussion section for those who want to review the topic.\n\nConvex optimization is a mathematical optimization technique used to find the minimum or maximum of a convex function over a convex set. It is commonly used in machine learning algorithms to optimize models and find the best parameters. The instructor mentions that they will discuss convex optimization in the discussion sections as an extension to the main lectures.\n\nHidden Markov models (HMMs) are statistical models used to model time series data, where the underlying system is assumed to be a Markov process with hidden states. HMMs are wide